In [ ]:
import os

import sys

sys.path.append("../..")

from src.logging_metrics import MetricsLogger

FOLDER = "../../metrics/logged_metrics/"

In [ ]:
def get_logs() -> list[MetricsLogger]:
    # remove useless files
    logs = [
        MetricsLogger.from_csv(os.path.join(FOLDER, filename))
        for filename in os.listdir(FOLDER)
        if not filename.startswith(".")
        and "considerws" not in filename
        and "consider_ws" not in filename
        and "autoencoders" not in filename
        and "aemlp" not in filename
        and "GP" not in filename
    ]

    # order by timestamp
    logs = sorted(logs, key=lambda x: x.timestamp)

    # remove repetitions (keeping the most recent one)
    already_added = set()
    new_logs = list()
    for log in logs[::-1]:
        if log.name in already_added:
            continue
        already_added.add(log.name)
        new_logs.append(log)

    return new_logs

In [ ]:
logs = get_logs()
interesting_logs_NN = list()
interesting_logs_NeRF = list()
names = []

In [ ]:
# INTERPOLATION - REDUCTION FACTOR 4
for log in logs:
    if (
        "ti4" in log.name
        and "xD" not in log.name
        and "64" not in log.name
        and not "multivariate" in log.name
    ):
        print(log.name)
        log.plot_metrics_by_epoch()
        if "NeRF" in log.name:
            interesting_logs_NeRF.append(log)
        elif "NN" in log.name:
            interesting_logs_NN.append(log)
names.append("Standard interpolation")

In [ ]:
# INTERPOLATION ON COORDS - REDUCTION FACTOR 4
for log in logs:
    if "xD4" in log.name and "64" not in log.name:
        print(log.name)
        log.plot_metrics_by_epoch()
        if "NeRF" in log.name:
            interesting_logs_NeRF.append(log)
        elif "NN" in log.name:
            interesting_logs_NN.append(log)
names.append("Interpolation on coords")

In [ ]:
# EXTRAPOLATION
for log in logs:
    if "ranges" in log.name and "multivariate" not in log.name and "64" not in log.name:
        print(log.name)
        log.plot_metrics_by_epoch()
        if "NeRF" in log.name:
            interesting_logs_NeRF.append(log)
        elif "NN" in log.name:
            interesting_logs_NN.append(log)
names.append("Extrapolation")

In [ ]:
# COMBINING
for log in interesting_logs_NeRF:
    print(log.name)
MetricsLogger.plot_many_logs_metrics_by_epoch(
    interesting_logs_NeRF, names=names
)  # , end_at_epoch=100)
print("\n\n")
for log in interesting_logs_NN:
    print(log.name)
MetricsLogger.plot_many_logs_metrics_by_epoch(interesting_logs_NN, names=names)